<a href="https://colab.research.google.com/github/SergeiVKalinin/MSE_Fall_2025/blob/main/Module%202/7_Matminer_perovskites_SISSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq matminer

In [ ]:
from matminer.datasets import load_dataset

df = load_dataset("castelli_perovskites")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
from matminer.featurizers.conversions import StrToComposition
df = StrToComposition().featurize_dataframe(df, "formula")
df.head()

In [ ]:
from matminer.featurizers.composition import ElementProperty

ep_feat = ElementProperty.from_preset(preset_name="magpie")
df = ep_feat.featurize_dataframe(df, col_id="composition")  # input the "composition" column to the featurizer
df.head()

In [ ]:
df.columns

In [ ]:
y = df['gap is direct'].values

excluded = ['fermi level', 'fermi width', 'e_form', 'gap is direct', 'structure',
       'mu_b', 'formula', 'vbm', 'cbm', 'gap gllbsc', 'composition']
X = df.drop(excluded, axis=1)

print("There are {} possible descriptors:\n\n{}".format(X.shape[1], X.columns.values))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Perform PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot the data
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[y, 0], X_pca[y, 1], c='blue', label='True')
plt.scatter(X_pca[~y, 0], X_pca[~y, 1], c='red', label='False')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('PCA of Data with gap is direct')
plt.legend()
plt.show()

In [ ]:
import xgboost as xgb
import pandas as pd

# Initialize and train the XGBoost classifier
model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
model.fit(X_train, y_train)

# Get feature importances
feature_importances = model.feature_importances_

# Create a pandas Series for easier handling
feature_importances = pd.Series(feature_importances, index=X.columns)

# Sort feature importances and get the top 10
top_10_features = feature_importances.sort_values(ascending=False).head(10)

print("Top 10 most significant features:")
print(top_10_features)

In [ ]:
import matplotlib.pyplot as plt

# Plot the top 10 feature importances as a bar graph
plt.figure(figsize=(10, 6))
top_10_features.plot(kind='bar')
plt.title('Top 10 Most Significant Features (XGBoost)')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_prob = model.predict_proba(X_test)[:, 1]

# Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'XGBoost (AUC = {auc_score:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for XGBoost Classifier')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
!pip -qq install TorchSisso scikit-learn matplotlib pandas sympy

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from TorchSisso import SissoModel
import sympy as sp

In [ ]:
y1 = df['gap is direct'].values

included = ['MagpieData maximum NUnfilled', 'MagpieData mean NpUnfilled', 'MagpieData mean Column',
            'MagpieData mean NUnfilled', 'MagpieData minimum Column', 'MagpieData avg_dev GSbandgap']
X1 = df[included]

print("There are {} possible descriptors:\n\n{}".format(X1.shape[1], X1.columns.values))

In [ ]:
y1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             roc_auc_score, f1_score, precision_score,
                             recall_score, confusion_matrix)
from TorchSisso import SissoModel
import sympy as sp

# --- INPUTS ---
# df: your DataFrame with the 6 columns below
cols = [
    'MagpieData maximum NUnfilled',
    'MagpieData mean NpUnfilled',
    'MagpieData mean Column',
    'MagpieData mean NUnfilled',
    'MagpieData minimum Column',
    'MagpieData avg_dev GSbandgap'
]
X = df[cols].copy()
y = np.asarray(y1, dtype=float)   # True/False -> 1.0/0.0

# --- CLEANUP ---
# keep only rows with no NaNs and valid y
mask = (~X.isna().any(axis=1)).values & ~np.isnan(y)
X = X.loc[mask].reset_index(drop=True)
y = y[mask]

# TorchSISSO is happier with simple column names
safe_cols = [f'x{i+1}' for i in range(X.shape[1])]
name_map = dict(zip(cols, safe_cols))
X_safe = X.rename(columns=name_map)

# --- SPLIT ---
strat = y if np.unique(y).size == 2 else None
Xtr, Xte, ytr, yte = train_test_split(
    X_safe, y, test_size=0.25, stratify=strat, random_state=0
)

# Build training DataFrame (TorchSISSO expects first column = target y)
train_df = pd.concat([pd.Series(ytr, name='y'), Xtr.reset_index(drop=True)], axis=1)
test_df  = pd.concat([pd.Series(yte, name='y'), Xte.reset_index(drop=True)], axis=1)

# --- TORCHSISSO ---
# Use a "safe" operator set (no raw ln to avoid domain issues)
operators = ['+','-','*','/','pow(2)','pow(3)','abs','sqrt']

sm = SissoModel(
    train_df,        # positional arg: df with y first, then features
    operators=operators,
    n_expansion=3,   # try 2–4
    n_term=2,        # 1–3 terms for interpretability
    k=50,            # SIS screening pool
    use_gpu=False
)

out = sm.fit()
try:
    rmse, equation, r2, final_eq = out
except ValueError:
    rmse, equation, r2 = out
    final_eq = None

print("TorchSISSO equation:", equation)
print("Train RMSE:", rmse, " | Train R^2:", r2)

In [ ]:
# --- MAKE THE EQUATION CALLABLE ---
sym_map = {s: sp.symbols(s) for s in safe_cols}
eq_str = str(equation).replace('ln(', 'log(').replace('pow(', 'Pow(').replace('^', '**')
expr = sp.sympify(eq_str, locals={
    **sym_map, 'sin': sp.sin, 'cos': sp.cos, 'sqrt': sp.sqrt, 'abs': sp.Abs,
    'exp': sp.exp, 'log': sp.log, 'Pow': sp.Pow
})
f = sp.lambdify(tuple(sym_map.values()), expr, modules={
    'numpy': np, 'sin': np.sin, 'cos': np.cos, 'sqrt': np.sqrt,
    'abs': np.abs, 'exp': np.exp, 'log': np.log, 'Pow': np.power
})

In [ ]:
# --- EVALUATE AS CLASSIFIER (threshold 0.5) ---
Xte_vals = [Xte[c].values for c in safe_cols]
yhat = np.asarray(f(*Xte_vals), dtype=float)
ypred = (yhat >= 0.5).astype(int)

print("Test accuracy      :", accuracy_score(yte, ypred))
print("Balanced accuracy  :", balanced_accuracy_score(yte, ypred))
try:
    print("ROC AUC (prob-like):", roc_auc_score(yte, yhat))
except Exception:
    pass
print("F1                 :", f1_score(yte, ypred))
print("Precision          :", precision_score(yte, ypred))
print("Recall             :", recall_score(yte, ypred))
print("Confusion matrix:\n", confusion_matrix(yte, ypred))

# Pretty-print equation using ORIGINAL feature names
pretty = str(expr)
for orig, safe in name_map.items():
    pretty = pretty.replace(safe, f'[{orig}]')
print("\nEquation with original names:\n", pretty)